In [ ]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import pandas as pd
import csv
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import integrate
from lmfit import minimize, Parameters, Parameter, report_fit
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from shapely.geometry import Polygon
from shapely.geometry import Point
import random

def meanvals(data_fileroot, size, cells):
    ''' Reads data from CSV, sorts, and renames columns'''
    fin = (data_fileroot+".csv")
    data=pd.read_csv(fin)
    data.rename(columns = {'S-Max. Principal (Abs)':'stress'}, inplace = True)
    data=data.drop(columns=['ODB Name' , 'Step' ,'Frame', 'Part Instance Name', 'Section Name', 'Material Name', 'Section Point'])
    data = data.sort_values(by=['X', 'Y','Z'], ascending=False)
    datamatrix=np.array(data)
    #############################################################################
    stressmatrix=np.empty((2016032,4))
    for n in range (1,22176352,11):
        stressmatrix[int((n-1)/11),0]=datamatrix[n,1]
        stressmatrix[int((n-1)/11),1]=datamatrix[n,2]
        stressmatrix[int((n-1)/11),2]=datamatrix[n,3]
        stressmatrix[int((n-1)/11),3]=np.average(datamatrix[n-1:n+9,4])
        
    #############################################################################
    for n in range (0,len(stressmatrix),32):
        n=n
    depthvalues=stressmatrix[n:n+32,2]
    
    #############################################################################
    lengthdivisions=np.sqrt(int(cells))
    temp=lengthdivisions-1
    bounds=np.empty((int(lengthdivisions-1),1))
    y=lengthdivisions-1
    while y>=1:
        bounds[int(y)-1]=(y/lengthdivisions)*size
        y=y-1
    #############################################################################
    binnumber=np.empty((len(stressmatrix),1))
    for n in range (0,len(stressmatrix),1):
        tempbin=np.empty((2,1))
        tempbin[0]=np.digitize(stressmatrix[n,0], bounds.flatten(),right=True)
        tempbin[1]=np.digitize(stressmatrix[n,1], bounds.flatten(),right=True)
        binnumbertemp=((tempbin[0])*(int(lengthdivisions))**0)+((tempbin[1])*(int(lengthdivisions))**1)
        binnumber[n]=int(binnumbertemp)


    stressmatrix_bins=np.append(stressmatrix,binnumber, axis=1)
    stressmatrix_bins=pd.DataFrame(stressmatrix_bins, columns=['X','Y','Z','stress','bins'])
    stressmatrix_bins=np.array(stressmatrix_bins.sort_values(by=['bins', 'Z','X','Y'], ascending=False))
    #############################################################################
    unique, counts = np.unique(stressmatrix_bins[:,4], return_counts=True)
    unique=np.flip(unique)
    counts=np.flip(counts)
    uniquedepths, countsdepths = np.unique(stressmatrix_bins[:,2], return_counts=True)
    #############################################################################
    
    runningsum=0
    binstressmatrix=np.empty((len(uniquedepths),len(unique)))
    for n in range (0,len(unique),1):
        temp_bins=np.empty((counts[n],1))
        temp_depths=np.empty((counts[n],1))
        #temp_bins=np.array([stressmatrix_bins.Z[runningsum-counts[n]:runningsum],stressmatrix_bins.stress[runningsum-counts[n]:runningsum]])
        temp_bins=stressmatrix_bins[runningsum:runningsum+counts[n]-1,3]
        temp_depths=stressmatrix_bins[runningsum:runningsum+counts[n]-1,2]
        uniquedepths, countsdepths = np.unique(temp_depths, return_counts=True)
        runningsum=runningsum+counts[n]
        runningrows=0
        for p in range (0,len(uniquedepths),1):
            binstressmatrix[p,n]=np.average(temp_bins[runningrows:runningrows+countsdepths[p]-1])
            runningrows=runningrows+countsdepths[p]
            
            
    #############################################################################

    fig, ax2 = plt.subplots(1, 1)

    crossoverpts=np.empty((len(unique),1))
    totalwork=np.empty((len(unique),1))
    bestvals=np.empty((len(unique),3))
    bestvals2=np.empty((len(unique),3))

    def mod_st_exp(x,c,tau,beta):
            return (c*(np.exp(1-(((x)/tau)**(x/beta)))))

    for r in range (0,len(unique),1):
       
        init_vals =[float(binstressmatrix[0,r]), 0.2, 5]

        bounds=((-10000,0.01,-10),(0,10,10))
        
        best_vals, covar = curve_fit(mod_st_exp, -(depthvalues), (binstressmatrix[:,r]), p0=init_vals, bounds=bounds, absolute_sigma=False, maxfev=150000)
        
        bestvals[r]=best_vals


        ax2.plot(-depthvalues, binstressmatrix[:,r], label='Simulation Data %(#)03d' % \
              { "#": r+1})
        ax2.plot(-depthvalues, mod_st_exp(-depthvalues,*best_vals), label="Mod. Stretch Exp. Fit", color='orange')
        ax2.set_xlabel('Depth (mm)')
        ax2.set_ylabel('Induced Stress (MPa)')
        ax2.set_title('Modified Exponential Fit')

    ax2.grid()
    print(bestvals)
    return(bestvals, stressmatrix)

def RSD(stressmatrix, size, cells):

    ''' Reads data from CSV, sorts, and renames columns'''
    for n in range (0,len(stressmatrix),32):
        n=n
    depthvalues=stressmatrix[n:n+32,2]
    
    #############################################################################
    lengthdivisions=np.sqrt(int(cells))
    temp=lengthdivisions-1
    bounds=np.empty((int(lengthdivisions-1),1))
    y=lengthdivisions-1
    while y>=1:
        bounds[int(y)-1]=(y/lengthdivisions)*size
        y=y-1
    #############################################################################
    binnumber=np.empty((len(stressmatrix),1))
    for n in range (0,len(stressmatrix),1):
        tempbin=np.empty((2,1))
        tempbin[0]=np.digitize(stressmatrix[n,0], bounds.flatten(),right=True)
        tempbin[1]=np.digitize(stressmatrix[n,1], bounds.flatten(),right=True)
        binnumbertemp=((tempbin[0])*(int(lengthdivisions))**0)+((tempbin[1])*(int(lengthdivisions))**1)
        binnumber[n]=int(binnumbertemp)


    stressmatrix_bins=np.append(stressmatrix,binnumber, axis=1)
    stressmatrix_bins=pd.DataFrame(stressmatrix_bins, columns=['X','Y','Z','stress','bins'])
    stressmatrix_bins=np.array(stressmatrix_bins.sort_values(by=['bins', 'Z','X','Y'], ascending=False))
    #############################################################################
    unique, counts = np.unique(stressmatrix_bins[:,4], return_counts=True)
    unique=np.flip(unique)
    counts=np.flip(counts)
    uniquedepths, countsdepths = np.unique(stressmatrix_bins[:,2], return_counts=True)
    #############################################################################
    
    runningsum=0
    binstressmatrix=np.empty((len(uniquedepths),len(unique)))
    for n in range (0,len(unique),1):
        temp_bins=np.empty((counts[n],1))
        temp_depths=np.empty((counts[n],1))
        #temp_bins=np.array([stressmatrix_bins.Z[runningsum-counts[n]:runningsum],stressmatrix_bins.stress[runningsum-counts[n]:runningsum]])
        temp_bins=stressmatrix_bins[runningsum:runningsum+counts[n]-1,3]
        temp_depths=stressmatrix_bins[runningsum:runningsum+counts[n]-1,2]
        uniquedepths, countsdepths = np.unique(temp_depths, return_counts=True)
        runningsum=runningsum+counts[n]
        runningrows=0
        for p in range (0,len(uniquedepths),1):
            binstressmatrix[p,n]=np.average(temp_bins[runningrows:runningrows+countsdepths[p]-1])
            runningrows=runningrows+countsdepths[p]
            
            
    #############################################################################

    #fig, ax2 = plt.subplots(1, 1)

    crossoverpts=np.empty((len(unique),1))
    totalwork=np.empty((len(unique),1))
    bestvals=np.empty((len(unique),3))
    bestvals2=np.empty((len(unique),3))

    def mod_st_exp(x,c,tau,beta):
            return (c*(np.exp(1-(((x)/tau)**(x/beta)))))

    for r in range (0,len(unique),1):
       
        init_vals =[float(binstressmatrix[0,r]), 0.2, 5]

        bounds=((-10000,0.01,-10),(0,10,10))
        
        best_vals, covar = curve_fit(mod_st_exp, -(depthvalues), (binstressmatrix[:,r]), p0=init_vals, bounds=bounds, absolute_sigma=False, maxfev=150000)
        
        bestvals[r]=best_vals


        '''
        ax2.plot(-depthvalues, binstressmatrix[:,r], label='Simulation Data %(#)03d' % \
              { "#": r+1})
        ax2.plot(-depthvalues, mod_st_exp(-depthvalues,*best_vals), label="Mod. Stretch Exp. Fit", color='orange')
        ax2.set_xlabel('Depth (mm)')
        ax2.set_ylabel('Induced Stress (MPa)')
        ax2.set_title('Modified Exponential Fit')
        '''

    #ax2.grid()
    #print(bestvals)
    return(np.std(bestvals[:,0]),np.std(bestvals[:,1]),np.std(bestvals[:,2]))


def RSD_analysis(data_fileroot, size, cells):
    meanvalues, stressmatrix1=meanvals(data_fileroot, size, 1)
    rsdvals=np.empty((cells,3))
    for n in range (1,cells,1):
        num=RSD(stressmatrix1, size, int(n**2))
        rsdvals[n-1,:]=num/meanvalues
        print(n)
    rsdvals=np.array(rsdvals)
    np.savetxt("{}}_RSD.csv".format(data_fileroot), rsdvals, delimiter=",")

def regression(data_fileroot):
    data=np.genfromtxt('{}_RSD.csv'.format(data_fileroot), delimiter=',')
    a=0
    while data[a,0] < 0 :
        a=a+1
    x_input=np.empty((a,1))
    for s in range (0,a,1):
        x_input[s]=1/s**2
    fit=np.empty((len(data),len(data[0])))
    regrmatrix=np.empty((2,len(data[0])))
    for n in range (0,len(data[0]),1):
        x=np.log10(abs(x_input))
        x=x.reshape(len(x),1)
        y=np.log10(abs(y_input[:,n]))
        y=y.reshape(len(y),1)
        regr = LinearRegression()
        regr.fit(x, y)
        regrmatrix[0,n]=regr.coef_
        regrmatrix[1,n]=regr.intercept_
        
        #prediction=relativex[1:len(relativex)]**(regrmatrix[0])
        fit[:,n]=(10**(regrmatrix[1,n]))*x_input.flatten()**(regrmatrix[0,n])
        #fit[:,n]=fit.reshape(len(prediction_stress_data65_04725),1)
        regr.coef_
    return(fit, regrmatrix)
#EDIT TO INCLUDE ROBUST INPUTS^
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_2.intersection(poly_1).area / poly_1.union(poly_2).area
    proportion=poly_2.area/(poly_1.union(poly_2).area)
    return (iou/proportion)
def impactdensity(name):
    data=pd.read_csv("{}.txt".format(name))
    gridx=np.linspace(0,5,100)
    gridy=np.linspace(0,5,100)
    impactsperpt=np.empty((99,99))
    for n in range (0,len(gridx)-1,1):
        for m in range (0,len(gridy)-1,1):
            runningtotal=0
            for p in range (0,len(data.x),1):
                box=[[gridx[n],gridy[m]],[gridx[n],gridy[m+1]],[gridx[n+1],gridy[m+1]],[gridx[n+1],gridy[m]]]
                circle=Point(data.x[p],data.y[p]).buffer(data.impactdiameter[p]/2)
                intersect=calculate_iou(box,circle)
                if intersect != 0:
                    runningtotal=runningtotal+1
            impactsperpt[m,n]=runningtotal

        print(n)
    plt.pcolormesh(impactsperpt, cmap='viridis_r', )
    plt.colorbar()
    plt.savefig('{}_densityplt'.format(data_fileroot))
    return(impactsperpt)

def impactplot(data_fileroot):
    data=pd.read_csv('{}.txt'.format(data_fileroot))
    data=data.sort_values('impactdiameter', ascending=False)
    plt.figure(figsize=(10,10))
    plt.scatter(data.x,data.y,s=np.pi*(72*data.impactdiameter/2)**2,edgecolors='black', linewidths=1)
    plt.xlim([0, 5])
    plt.ylim([0, 5])
    plt.grid()
    plt.title('Impact Map, {}'.format(data_fileroot))
    plt.savefig('{}_impactmap'.format(data_fileroot))



In [5]:
data_fileroot="test"
'{}_RSD.txt'.format(data_fileroot)

'test_RSD.txt'